# Quantum Engine Reservation Utility

This Colab provides canned interactions with Cirq to manage a project's reservations on the Quantum Engine.

For information on how to download a colab from github, see [these instructions](colab.ipynb).  You can also [view this file](https://github.com/quantumlib/Cirq/blob/master/docs/tutorials/google/reservations.ipynb) or download the [raw content](https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/tutorials/google/reservations.ipynb) of this content from Github.

## Configure.
Choose the project to manage, autheticate, and install the necessary tools.

In [ ]:
def setup_auth():
  """Runs the user through the Colab OAuth process to set the local Application
  Default Credentials.
  
  For more information on on using Application Default Credentials see 
  https://cloud.google.com/docs/authentication/production
  """
  from google.colab import auth
  auth.authenticate_user(clear_output=False)

print("Getting OAuth2 credentials.")
print("Press enter after entering the verification code.")
setup_auth()
print("Installing cirq.")
#!pip install cirq
!pip install git+https://github.com/quantumlib/cirq.git --quiet

import cirq
from cirq.google.engine.client.quantum_v1alpha1.gapic import enums

# The Google Cloud Project id to use.
project_id = '' #@param {type:"string"}
# The processor to view.
processor_id = 'mcgee' #@param ['rainbow', 'pacific', 'mcgee']
# The local time zone.
time_zone = 'America/Los_Angeles'  #@param {type:"string"}

engine = cirq.google.Engine(project_id, proto_version=cirq.google.ProtoVersion.V2)
processor = engine.get_processor(processor_id)

import datetime
import pytz
tz = pytz.timezone(time_zone)
now = tz.localize(datetime.datetime.now())

def date_string(timestamp):
  if timestamp.seconds < 0:
    return 'Beginning of time'
  if timestamp.seconds > 4771848621:
    return 'End of time'
  time = datetime.datetime.fromtimestamp(timestamp.seconds).astimezone(tz)
  return time.strftime('%Y-%m-%d %H:%M:%S')
  
def delta(start, end):
  if start.seconds < 0 or end.seconds > 5000000000:
    return "∞"
  return "{} hrs".format((end.seconds - start.seconds) / (60 * 60))

def time_slot_string(time_slot):
  start = date_string(time_slot.start_time)
  end = date_string(time_slot.end_time)
  slot_type = cirq.google.engine.client.quantum_v1alpha1.types.QuantumTimeSlot.TimeSlotType.Name(time_slot.slot_type)
  slot_string = "{} to {} ({}) - {}".format(start, end, delta(time_slot.start_time, time_slot.end_time), slot_type)
  if time_slot.HasField('reservation_config'):
    return "{} for {}".format(slot_string, time_slot.reservation_config.project_id) 
  if time_slot.HasField('maintenance_config'):
    return "{} {} - {}".format(slot_string, time_slot.maintenance_config.title, time_slot.maintenance_config.description) 
  return slot_string

def reservation_string(reservation):
  start = date_string(reservation.start_time)
  end = date_string(reservation.end_time)
  id = reservation.name.split('/reservations/')[1]
  reservation_string = "{} to {} ({}) - {}".format(start, end, delta(reservation.start_time, reservation.end_time), id)
  if len(reservation.whitelisted_users) > 0:
    return "{} {}".format(reservation_string, reservation.whitelisted_users)
  return reservation_string 

print("============================")
print("Runtime setup completed")

# Checkout the upcoming schedule

In [ ]:
#@title
schedule = processor.get_schedule()
for s in schedule:
  print(time_slot_string(s))

## Find available time.

In [ ]:
#@title
schedule = processor.get_schedule()
unallocated = list(filter(lambda t: t.slot_type == enums.QuantumTimeSlot.TimeSlotType.UNALLOCATED, schedule))
for s in unallocated:
  print(time_slot_string(s))

if len(unallocated) == 0:
  print("No available time slots")

# List upcoming reservations for the project

In [ ]:
#@title
reservations = processor.list_reservations()

for r in reservations:
  print(reservation_string(r))

# Reserve time.

In [ ]:
#@markdown Create a new reservation for the given start date and time with the given duration in hours.
start_date_picker = "2020-07-30" #@param {type:"date"}
start_time = "12" #@param [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
hours =  2#@param {type:"integer"}
#@markdown Comma-separated email addresses of any additional users to explicitly whitelist for the reservation.
whitelist_user_emails = "" #@param {type:"string"}

start_time_naive = datetime.datetime.strptime("{} {}".format(start_date_picker, start_time), '%Y-%m-%d %H')
start_time = tz.localize(start_time_naive)
end_time = start_time + datetime.timedelta(hours=hours)

print(reservation_string(processor.create_reservation(start_time=start_time,
                                   end_time=end_time,
                                   whitelisted_users=[e.strip() for e in whitelist_user_emails.split(',') if e])))

# Update reservation time.

In [ ]:
#@markdown Update the deatils of an existing reservation. _You can find the `reservation_id` by listing your reservations or checking the output when you create a new reservation._
reservation_id = "" #@param {type:"string"}
start_date_picker = "2020-03-27" #@param {type:"date"}
start_time = "12" #@param [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]
hours =  2#@param {type:"integer"}

start_time_naive = datetime.datetime.strptime("{} {}".format(start_date_picker, start_time), '%Y-%m-%d %H')
start_time = tz.localize(start_time_naive)
end_time = start_time + datetime.timedelta(hours=hours)

print(reservation_string(processor.update_reservation(reservation_id=reservation_id,
                                   start_time=start_time,
                                   end_time=end_time)))

# Update reservation whitelisted users.

In [ ]:
#@markdown Update the deatils of an existing reservation. _You can find the `reservation_id` by listing your reservations or checking the output when you create a new reservation._
reservation_id = "" #@param {type:"string"}
#@markdown Comma-separated email addresses of any additional users to explicitly whitelist for the reservation.
whitelisted_user_emails = "" #@param {type:"string"}

print(reservation_string(processor.update_reservation(reservation_id=reservation_id,
                                   whitelisted_users=[e.strip() for e in whitelisted_user_emails.split(',') if e])))

# Remove reservation.

In [ ]:
#@markdown Delete a specific reservation as long as it is outside the schedule freeze. Inside the schedule freeze period reservations are cancelled instead.
reservation_id = "" #@param {type:"string"}

processor.remove_reservation(reservation_id)